## 0. Getting setup

In [52]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms

print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

try:
    from torchinfo import summary
except:
    print("[INFO] Coudnl't find torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts...")


torch version: 2.2.2
torchvision version: 0.17.2


In [53]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [54]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

# 1. Get data

In [55]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data/pizza_steak_sushi directory exists, skipping download.


PosixPath('data/pizza_steak_sushi')

# 2. Create Datasets and DataLoaders

In [56]:
train_dir = image_path / "train"
test_dir = image_path / "test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.226])

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize])
print(f"Manually created transforms: {manual_transforms}")

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.226])
)


(<torch.utils.data.dataloader.DataLoader at 0x7ef76d19e410>,
 ['pizza', 'steak', 'sushi'])

In [57]:
model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # for torchvision < 13
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT for torchvision > 0.13
# model = torchvision.models.efficientnet_b0(weights=weights)

In [58]:
# Freeze all base layers
for param in model.features.parameters():
    param.requires_grad = False

set_seeds()

model.classfier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names),
              bias=True).to(device))

In [59]:
# from torchinfo import summary

# summary(model,
#         input_size=(32, 3, 224, 224),
#         verbose=0,
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

# 4. Train model and track results

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [61]:
try:
    import wandb
except:
    !pip install wandb
    import wandb

In [62]:
from typing import Dict, List
from tqdm.auto import tqdm
from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          criterion: torch.nn.Module,
          epochs: int,
          device: torch.device,
         wandb_project_name: str = None,
         wandb_run_name: str = None) -> Dict[str, List]:
    
    if wandb_project_name is not None:
        wandb.init(project=wandb_project_name, name=wandb_project_name)
    
    results = {"train_loss": [],
               "test_loss": [],
               "train_acc": [],
               "test_acc": []
              }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=test_dataloader,
                                           loss_fn=criterion,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                         dataloader=test_dataloader,
                                         loss_fn=criterion,
                                         device=device)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_acc: {test_acc:.4f} | "
        )

        if wandb_project_name is not None:
            wandb.log({"train_loss": train_loss,
            "test_loss": test_loss,
            "train_acc": train_acc,
            "test_acc": test_acc})
            
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["train_acc"].append(train_acc)
        results["test_acc"].append(test_acc)

    return results

In [63]:
set_seeds()
model0 = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=20,
                device=device,
               wandb_project_name="torch_experiment_track",
               wandb_run_name="torch_experiment_v1")

wandb: Currently logged in as: frackowiak (frackowiak_kajetan). Use `wandb login --relogin` to force relogin


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 7.6200 | test_loss: 8.8285 | train_acc: 0.0000 | test_acc: 0.0000 | 
Epoch: 2 | train_loss: 6.5586 | test_loss: 7.3306 | train_acc: 0.0000 | test_acc: 0.0000 | 
Epoch: 3 | train_loss: 5.8622 | test_loss: 5.8943 | train_acc: 0.0000 | test_acc: 0.0000 | 
Epoch: 4 | train_loss: 4.9484 | test_loss: 4.7190 | train_acc: 0.1023 | test_acc: 0.0720 | 
Epoch: 5 | train_loss: 4.1468 | test_loss: 3.8594 | train_acc: 0.1544 | test_acc: 0.1544 | 
Epoch: 6 | train_loss: 3.5626 | test_loss: 3.2013 | train_acc: 0.1544 | test_acc: 0.1752 | 
Epoch: 7 | train_loss: 2.9876 | test_loss: 2.7624 | train_acc: 0.2879 | test_acc: 0.2576 | 
Epoch: 8 | train_loss: 2.5725 | test_loss: 2.4692 | train_acc: 0.3797 | test_acc: 0.3087 | 
Epoch: 9 | train_loss: 2.2193 | test_loss: 2.2241 | train_acc: 0.4318 | test_acc: 0.3400 | 
Epoch: 10 | train_loss: 2.0125 | test_loss: 2.0421 | train_acc: 0.4318 | test_acc: 0.4621 | 
Epoch: 11 | train_loss: 1.8522 | test_loss: 1.8956 | train_acc: 0.4299 | test_a

In [64]:
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data/pizza_steak_sushi directory exists, skipping download.
[INFO] data/pizza_steak_sushi_20_percent directory exists, skipping download.


In [65]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

test_dir = data_10_percent_path / "test"

print(f"Training_directory 10%: {train_dir_10_percent}")
print(f"Training_directory 20%: {train_dir_20_percent}")
print(f"Testing_directory {test_dir}")

Training_directory 10%: data/pizza_steak_sushi/train
Training_directory 20%: data/pizza_steak_sushi_20_percent/train
Testing_directory data/pizza_steak_sushi/test


In [66]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [67]:
BATCH_SIZE = 32

train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir_10_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir_20_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(test_dataloader)} (all experiments will use the same test data)")
print(f"Number of classes {len(class_names)}, class_names {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 3 (all experiments will use the same test data)
Number of classes 3, class_names ['pizza', 'steak', 'sushi']


In [68]:
import torchvision
from torchinfo import summary

effnetb2 = torchvision.models.efficientnet_b2(pretrained=True).to(device)

# summary(model=effnetb2,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

print(f"Number of in_features to final layer of EfficientNetB2; {len(effnetb2.classifier.state_dict()['1.weight'][0])}")

/home/kajetan/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kajetan/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of in_features to final layer of EfficientNetB2; 1408


In [69]:
import torchvision
from torch import nn

OUT_FEATURES = len(class_names)

def create_effnetb0():
    model = torchvision.models.efficientnet_b0(pretrained=True).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)

    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2():
    model = torchvision.models.efficientnet_b2(pretrained=True).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classfier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)

    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [70]:
effnetb0 = create_effnetb0()

# summary(model=effnetb0,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

[INFO] Created new effnetb0 model.


/home/kajetan/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [71]:
effnetb2 = create_effnetb2()

# summary(model=effnetb2,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

[INFO] Created new effnetb2 model.


In [72]:
num_epochs = [5, 10]
models = ["effnetb0", "effnetb2"]
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [73]:
# %%time
# from going_modular.utils import save_model

# set_seeds(seed=42)

# experiment_number = 0

# for dataloader_name, train_dataloader in train_dataloaders.items():
#     for epochs in num_epochs:
#         for model_name in models:
#             experiment_number += 1
#             print(f"[INFO] Experiment number: {experiment_number}")
#             print(f"[INFO] Model: {model_name}")
#             print(f"[INFO] DataLoader: {dataloader_name}")
#             print(f"[INFO] Number of epochs: {epochs}")

#             if model_name == "effnetb0":
#                 model = create_effnetb0()
#             else:
#                 model = create_effnetb2()

#             criterion = nn.CrossEntropyLoss()
#             optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

#             train(model=model,
#                 train_dataloader=test_dataloader,
#                 test_dataloader=test_dataloader,
#                 optimizer=optimizer,
#                 criterion=criterion,
#                 epochs=epochs,
#                 device=device,
#                 wandb_project_name=model_name,
#                 wandb_run_name=model_name)

#             save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
#             save_model(model=model,
#                         target_dir="models",
#                         model_name=save_filepath)
#             print("-"*50+"\n")

In [74]:
train(model=effnetb0,
      train_dataloader=train_dataloader,
      test_dataloader=test_dataloader,
      optimizer=torch.optim.Adam(params=effnetb0.parameters(), lr=1e-2),
      criterion=criterion,
      epochs=25,
      device=device,
      wandb_project_name="effnetb0v1")

test_acc,▁▁▁▂▃▃▄▄▅▆▆▇▇▇▇█████
test_loss,█▇▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▂▃▃▄▅▆▆▆▆▇▇▇█▇█▇▇
train_loss,█▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁
test_acc,0.64867
test_loss,1.39679
train_acc,0.5464
train_loss,1.19573


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.4503 | test_loss: 0.8755 | train_acc: 0.0625 | test_acc: 0.6013 | 
Epoch: 2 | train_loss: 0.5073 | test_loss: 0.8259 | train_acc: 0.8551 | test_acc: 0.6250 | 
Epoch: 3 | train_loss: 1.1038 | test_loss: 0.6423 | train_acc: 0.6250 | test_acc: 0.6979 | 
Epoch: 4 | train_loss: 0.9805 | test_loss: 0.3882 | train_acc: 0.6884 | test_acc: 0.8655 | 
Epoch: 5 | train_loss: 0.7653 | test_loss: 0.4030 | train_acc: 0.5388 | test_acc: 0.8883 | 
Epoch: 6 | train_loss: 0.5729 | test_loss: 0.2458 | train_acc: 0.7339 | test_acc: 1.0000 | 
Epoch: 7 | train_loss: 0.3167 | test_loss: 0.2238 | train_acc: 0.9479 | test_acc: 0.9271 | 
Epoch: 8 | train_loss: 0.3818 | test_loss: 0.2105 | train_acc: 0.7708 | test_acc: 0.9167 | 
Epoch: 9 | train_loss: 0.4800 | test_loss: 0.1466 | train_acc: 0.7812 | test_acc: 1.0000 | 
Epoch: 10 | train_loss: 0.3921 | test_loss: 0.1499 | train_acc: 0.8854 | test_acc: 1.0000 | 
Epoch: 11 | train_loss: 0.3328 | test_loss: 0.1357 | train_acc: 0.9375 | test_a

{'train_loss': [2.4502642154693604,
  0.5072961747646332,
  1.1038346538941066,
  0.9804738412300745,
  0.765303244193395,
  0.5729138255119324,
  0.3166517565647761,
  0.3818257947762807,
  0.4800354490677516,
  0.3920569171508153,
  0.33276456594467163,
  0.2734677344560623,
  0.23961861928304037,
  0.25652217864990234,
  0.25063203523556393,
  0.2505962550640106,
  0.22769281268119812,
  0.14345781256755194,
  0.17087763672073683,
  0.15508605043093363,
  0.1581234261393547,
  0.155532938738664,
  0.12287664910157521,
  0.1260321040948232,
  0.1556467500825723],
 'test_loss': [0.8755178252855936,
  0.8259320010741552,
  0.6422924523552259,
  0.3882146974404653,
  0.4030019889275233,
  0.2458317130804062,
  0.22384149581193924,
  0.21052697052558264,
  0.14659111822644869,
  0.14989589899778366,
  0.13571483890215555,
  0.1121266521513462,
  0.1041627787053585,
  0.09454988191525142,
  0.09199534108241399,
  0.08959916730721791,
  0.07888854170838992,
  0.07399249883989494,
  0.06961

In [75]:
def load_model(filepath):
    checkpoint = torch.load(filepath)

    if checkpoint['model_name'] == "effnetb0V1":
        model = create_effnetb0()  # This line creates the model architecture
    elif checkpoint['model_name'] == "effnetb0V2":
        model = create_effnetb2()  # This line creates the model architecture
    else:
        raise ValueError("Unsupported model name")

    model.load_state_dict(checkpoint['model_state_dict'])  # This line loads the trained parameters

    print(f"[INFO] Model '{checkpoint['model_name']}' loaded successfully")
    return model


In [76]:
def load_model(filepath):
    checkpoint = torch.load(filepath)

    if checkpoint['model_name'] == "effnetb0V1":
        model = create_effnetb0()
    elif checkpoint['model_name'] == "effnetb0V2":
        model = create_effnetb2()
    else:
        raise ValueError("Unsupported model name")

    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"[INFO] Model '{checkpoint['model_name']}' loaded successfully")
    return model

# effnetV1_model_path = "models/effnetb0V1.pth"
# best_model = load_model(effnetV1_model_path)

In [77]:
# Check the model file size
from pathlib import Path

effnetb0_model_size = Path(effnetV1_model_path).stat().st_size // (1024*1024)
print(f"Effnetb0V1_loaded feature extractor model size: {effnetb0_model_size} MB")

FileNotFoundError: [Errno 2] No such file or directory: 'models/bonurke.pth'

In [ ]:
# from going_modular.predictions import pred_and_plot_image

# # Get a random list of 3 images from 20% test set
# import random
# num_images_to_plot = 3
# test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg")) # get all test image paths from 20% dataset
# test_image_path_sample = random.sample(population=test_image_path_list,
#                                        k=num_images_to_plot) # randomly select k number of images

# # Iterate through random test image paths, make predictions on them and plot them
# for image_path in test_image_path_sample:
#     pred_and_plot_image(model=effnetV1_loaded,
#                         image_path=image_path,
#                         class_names=class_names,
#                         image_size=(224, 224))


In [ ]:
# # Download custom image
# import requests

# # Setup custom image path
# custom_image_path = Path("data/04-pizza-dad.jpeg")

# # Download the image if it doesn't already exist
# if not custom_image_path.is_file():
#     with open(custom_image_path, "wb") as f:
#         # When downloading from GitHub, need to use the "raw" file link
#         request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
#         print(f"Downloading {custom_image_path}...")
#         f.write(request.content)
# else:
#     print(f"{custom_image_path} already exists, skipping download.")

# # Predict on custom image
# pred_and_plot_image(model=effnetV1_loaded,
#                     image_path=custom_image_path,
#                     class_names=class_names)